In [ ]:
import os

os.chdir("/home/jovyan/work/")

from neo4j_helper import Neo4jHelper, GraphLayout


neo4j = Neo4jHelper.setup_movie_graph()

### Match with OR labels

In [ ]:
neo4j.cypher("""
    MATCH p=(:Person)-[:ACTED_IN|FOLLOWS|REVIEWED]->(u)
    WHERE u:Movie OR u:Person
    RETURN p
""").show(layout=GraphLayout.RADIAL)

### Directors who acted in their own movies

In [ ]:
neo4j.cypher("""
    MATCH (p:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(p)
    RETURN p.name, m.title
""").execute_for_dataframe()

### Co-Actors list by movie

In [ ]:
neo4j.cypher("""
    MATCH (m:Movie)<-[:ACTED_IN]-(a)
    RETURN m.title, collect(a.name) AS perormers
""").execute_for_dataframe()

### Connect actors as colleagues if they acted in the same movie
add or append the movie as a property to the relationship.

In [ ]:
neo4j.cypher("""
    MATCH (m:Movie)<-[:ACTED_IN]-(a1:Person)
    MATCH (m)<-[:ACTED_IN]-(a2:Person)
    WHERE id(a1) < id(a2)
    MERGE (a1)-[c:COLLEAGUE]->(a2)
    ON CREATE SET c.movies = [m.title]
    ON MATCH SET c.movies = c.movies + m.title
""").execute()

In [ ]:
neo4j.show_all(layout=GraphLayout.RADIAL)

### Fetch colleagues with more than two shared movies

In [ ]:
neo4j.cypher("""
    MATCH (p:Person)-[c:COLLEAGUE]-(p2:Person)
    WHERE size(c.movies) > 2
    MATCH (p)-[a:ACTED_IN]-> (m:Movie)<-[a2:ACTED_IN]-(p2)
    WHERE m.title IN c.movies
    return *
""").execute_for_graph()

In [ ]:
neo4j.cypher("""
    MATCH p=(:Person)-[:COLLEAGUE]->(:Person)
    return p
""").show(layout=GraphLayout.RADIAL)